In [11]:
import networkx as nx
import matplotlib.pyplot as plt

filename = "top250movies.txt"

In [12]:
try:
    with open(filename, encoding="utf-8") as f:
        data = []
        for line in f:
            data.append(line.strip().split("/"))
except FileNotFoundError:
    raise IOError(f"Error: Could not find file {filename}")
except ValueError:
    raise ValueError(f"Error: Invalid data in file {filename}")
